In [1]:
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np
from trl import RewardTrainer, RewardConfig
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
REPO = "HuggingFaceTB/SmolLM2-135M-Instruct"
snapshot_download(repo_id=REPO, local_dir="./model")

In [2]:
from data_preprocess import process_pairs_dataset
from config import MODEL_PATH, DATA_PATH, SEED, SCALAR_REWARD_MODEL_PATH

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
reward_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=1,
    local_files_only=True,
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
reward_model.config.pad_token_id = reward_model.config.eos_token_id

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ./model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
train_data = load_dataset(DATA_PATH, split="train")
val_data = load_dataset(DATA_PATH, split="validation")

train_data = process_pairs_dataset(train_data)
val_data = process_pairs_dataset(val_data)

In [7]:
training_args = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    learning_rate=5e-5,
    num_train_epochs=1,
    logging_strategy="steps",
    logging_steps=300,
    eval_strategy="steps",
    eval_steps=300,
    save_steps=300,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none",
)

trainer = RewardTrainer(
    model=reward_model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

Для обучение используется стандартная функция потерь $$\mathcal{L}_{RM} = -\log \sigma\left(r_{\theta}(x, y_w) - r_{\theta}(x, y_l)\right)$$

In [8]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
300,0.754200,0.752164,0.594030
600,0.723800,0.688678,0.607784
900,0.688400,0.676795,0.634731
1200,0.654700,0.673410,0.616766


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>system                            │ <|im_start|>system                           │ [0.5763, 0.4237] │
│ You are a helpful AI assistant named SmolLM,  │ You are a helpful AI assistant named SmolLM, │                  │
│ trained by Hugging Face                       │ trained by Hugging Face                      │                  │
│ <|im_start|>user                              │ <|im_start|>user                             │                  │
│                                               │                                              │                  │
│  Is it possible to host data on a satellite?  │  Is it possible to host data on a satellite? │                  │
│ <extra_id_1>Assistant                         │ <extra_id_1>Assistant                        │                  │
│ Yes, it’s possible to host data on a          │ Yes, it’s possible to host data on a         │                  │
│ satellite. Satellites can carry computer      │ satellite. Satellites can carry computer     │                  │
│ servers and other electronic equipment, so    │ servers and other electronic equipment, so   │                  │
│ it’s possible to store and transmit data via  │ it’s possible to store and transmit data via │                  │
│ satellite. This is often used for remote data │ satellite. This is often used for remote     │                  │
│ storage and backup, as well as for internet   │ data storage and backup, as well as for      │                  │
│ and telecommunications services.              │ internet and telecommunications services.    │                  │
│ Satellite-hosted data is relatively secure,   │ Satellite-hosted data is relatively secure,  │                  │
│ as it’s not vulnerable to terrestrial threats │ as it’s not vulnerable to terrestrial        │                  │
│ like natural disasters or hacking, and it can │ threats like natural disasters or hacking,   │                  │
│ be accessed from anywhere on the globe.       │ and it can be accessed from anywhere on the  │                  │
│ <extra_id_1>User                              │ globe.                                       │                  │
│ Does hosting data on a satellite ensure that  │ <extra_id_1>User                             │                  │
│ governments and corporations will be unable   │ Does hosting data on a satellite ensure that │                  │
│ to acess the servers?                         │ governments and corporations will be unable  │                  │
│ <extra_id_1>Assistant                         │ to acess the servers?                        │                  │
│ Hosting data on a satellite does help ensure  │ <extra_id_1>Assistant                        │                  │
│ privacy and security, as satellite data       │ Hosting data on a satellite does help ensure │                  │
│ hosting services are often located in         │ privacy and security, as satellite data      │                  │
│ jurisdictions with strong privacy laws, and   │ hosting services are often located in        │                  │
│ are physically isolated from terrestrial      │ jurisdictions with strong privacy laws, and  │                  │
│ threats like hacking and natural disasters.   │ are physically isolated from terrestrial     │                  │
│ However, there is still some risk of          │ threats like hacking and natural disasters.  │                  │
│ unauthorized access to satellite-hosted data, │ However, there is still some risk of         │                  │
│ as satellites are vulnerable to attack from   │ unauth

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>system                            │ <|im_start|>system                           │ [0.7483, 0.2517] │
│ You are a helpful AI assistant named SmolLM,  │ You are a helpful AI assistant named SmolLM, │                  │
│ trained by Hugging Face                       │ trained by Hugging Face                      │                  │
│ <|im_start|>user                              │ <|im_start|>user                             │                  │
│                                               │                                              │                  │
│  Is it possible to host data on a satellite?  │  Is it possible to host data on a satellite? │                  │
│ <extra_id_1>Assistant                         │ <extra_id_1>Assistant                        │                  │
│ Yes, it’s possible to host data on a          │ Yes, it’s possible to host data on a         │                  │
│ satellite. Satellites can carry computer      │ satellite. Satellites can carry computer     │                  │
│ servers and other electronic equipment, so    │ servers and other electronic equipment, so   │                  │
│ it’s possible to store and transmit data via  │ it’s possible to store and transmit data via │                  │
│ satellite. This is often used for remote data │ satellite. This is often used for remote     │                  │
│ storage and backup, as well as for internet   │ data storage and backup, as well as for      │                  │
│ and telecommunications services.              │ internet and telecommunications services.    │                  │
│ Satellite-hosted data is relatively secure,   │ Satellite-hosted data is relatively secure,  │                  │
│ as it’s not vulnerable to terrestrial threats │ as it’s not vulnerable to terrestrial        │                  │
│ like natural disasters or hacking, and it can │ threats like natural disasters or hacking,   │                  │
│ be accessed from anywhere on the globe.       │ and it can be accessed from anywhere on the  │                  │
│ <extra_id_1>User                              │ globe.                                       │                  │
│ Does hosting data on a satellite ensure that  │ <extra_id_1>User                             │                  │
│ governments and corporations will be unable   │ Does hosting data on a satellite ensure that │                  │
│ to acess the servers?                         │ governments and corporations will be unable  │                  │
│ <extra_id_1>Assistant                         │ to acess the servers?                        │                  │
│ Hosting data on a satellite does help ensure  │ <extra_id_1>Assistant                        │                  │
│ privacy and security, as satellite data       │ Hosting data on a satellite does help ensure │                  │
│ hosting services are often located in         │ privacy and security, as satellite data      │                  │
│ jurisdictions with strong privacy laws, and   │ hosting services are often located in        │                  │
│ are physically isolated from terrestrial      │ jurisdictions with strong privacy laws, and  │                  │
│ threats like hacking and natural disasters.   │ are physically isolated from terrestrial     │                  │
│ However, there is still some risk of          │ threats like hacking and natural disasters.  │                  │
│ unauthorized access to satellite-hosted data, │ However, there is still some risk of         │                  │
│ as satellites are vulnerable to attack from   │ unauth

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>system                            │ <|im_start|>system                           │ [0.6556, 0.3444] │
│ You are a helpful AI assistant named SmolLM,  │ You are a helpful AI assistant named SmolLM, │                  │
│ trained by Hugging Face                       │ trained by Hugging Face                      │                  │
│ <|im_start|>user                              │ <|im_start|>user                             │                  │
│                                               │                                              │                  │
│  Is it possible to host data on a satellite?  │  Is it possible to host data on a satellite? │                  │
│ <extra_id_1>Assistant                         │ <extra_id_1>Assistant                        │                  │
│ Yes, it’s possible to host data on a          │ Yes, it’s possible to host data on a         │                  │
│ satellite. Satellites can carry computer      │ satellite. Satellites can carry computer     │                  │
│ servers and other electronic equipment, so    │ servers and other electronic equipment, so   │                  │
│ it’s possible to store and transmit data via  │ it’s possible to store and transmit data via │                  │
│ satellite. This is often used for remote data │ satellite. This is often used for remote     │                  │
│ storage and backup, as well as for internet   │ data storage and backup, as well as for      │                  │
│ and telecommunications services.              │ internet and telecommunications services.    │                  │
│ Satellite-hosted data is relatively secure,   │ Satellite-hosted data is relatively secure,  │                  │
│ as it’s not vulnerable to terrestrial threats │ as it’s not vulnerable to terrestrial        │                  │
│ like natural disasters or hacking, and it can │ threats like natural disasters or hacking,   │                  │
│ be accessed from anywhere on the globe.       │ and it can be accessed from anywhere on the  │                  │
│ <extra_id_1>User                              │ globe.                                       │                  │
│ Does hosting data on a satellite ensure that  │ <extra_id_1>User                             │                  │
│ governments and corporations will be unable   │ Does hosting data on a satellite ensure that │                  │
│ to acess the servers?                         │ governments and corporations will be unable  │                  │
│ <extra_id_1>Assistant                         │ to acess the servers?                        │                  │
│ Hosting data on a satellite does help ensure  │ <extra_id_1>Assistant                        │                  │
│ privacy and security, as satellite data       │ Hosting data on a satellite does help ensure │                  │
│ hosting services are often located in         │ privacy and security, as satellite data      │                  │
│ jurisdictions with strong privacy laws, and   │ hosting services are often located in        │                  │
│ are physically isolated from terrestrial      │ jurisdictions with strong privacy laws, and  │                  │
│ threats like hacking and natural disasters.   │ are physically isolated from terrestrial     │                  │
│ However, there is still some risk of          │ threats like hacking and natural disasters.  │                  │
│ unauthorized access to satellite-hosted data, │ However, there is still some risk of         │                  │
│ as satellites are vulnerable to attack from   │ unauth

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>system                            │ <|im_start|>system                           │ [0.6436, 0.3564] │
│ You are a helpful AI assistant named SmolLM,  │ You are a helpful AI assistant named SmolLM, │                  │
│ trained by Hugging Face                       │ trained by Hugging Face                      │                  │
│ <|im_start|>user                              │ <|im_start|>user                             │                  │
│                                               │                                              │                  │
│  Is it possible to host data on a satellite?  │  Is it possible to host data on a satellite? │                  │
│ <extra_id_1>Assistant                         │ <extra_id_1>Assistant                        │                  │
│ Yes, it’s possible to host data on a          │ Yes, it’s possible to host data on a         │                  │
│ satellite. Satellites can carry computer      │ satellite. Satellites can carry computer     │                  │
│ servers and other electronic equipment, so    │ servers and other electronic equipment, so   │                  │
│ it’s possible to store and transmit data via  │ it’s possible to store and transmit data via │                  │
│ satellite. This is often used for remote data │ satellite. This is often used for remote     │                  │
│ storage and backup, as well as for internet   │ data storage and backup, as well as for      │                  │
│ and telecommunications services.              │ internet and telecommunications services.    │                  │
│ Satellite-hosted data is relatively secure,   │ Satellite-hosted data is relatively secure,  │                  │
│ as it’s not vulnerable to terrestrial threats │ as it’s not vulnerable to terrestrial        │                  │
│ like natural disasters or hacking, and it can │ threats like natural disasters or hacking,   │                  │
│ be accessed from anywhere on the globe.       │ and it can be accessed from anywhere on the  │                  │
│ <extra_id_1>User                              │ globe.                                       │                  │
│ Does hosting data on a satellite ensure that  │ <extra_id_1>User                             │                  │
│ governments and corporations will be unable   │ Does hosting data on a satellite ensure that │                  │
│ to acess the servers?                         │ governments and corporations will be unable  │                  │
│ <extra_id_1>Assistant                         │ to acess the servers?                        │                  │
│ Hosting data on a satellite does help ensure  │ <extra_id_1>Assistant                        │                  │
│ privacy and security, as satellite data       │ Hosting data on a satellite does help ensure │                  │
│ hosting services are often located in         │ privacy and security, as satellite data      │                  │
│ jurisdictions with strong privacy laws, and   │ hosting services are often located in        │                  │
│ are physically isolated from terrestrial      │ jurisdictions with strong privacy laws, and  │                  │
│ threats like hacking and natural disasters.   │ are physically isolated from terrestrial     │                  │
│ However, there is still some risk of          │ threats like hacking and natural disasters.  │                  │
│ unauthorized access to satellite-hosted data, │ However, there is still some risk of         │                  │
│ as satellites are vulnerable to attack from   │ unauth

TrainOutput(global_step=1204, training_loss=0.7051763892965855, metrics={'train_runtime': 532.9282, 'train_samples_per_second': 13.555, 'train_steps_per_second': 2.259, 'total_flos': 0.0, 'train_loss': 0.7051763892965855, 'epoch': 1.0})

In [9]:
trainer.save_model(SCALAR_REWARD_MODEL_PATH)